In [30]:
from pprint import pprint
import json

def add_dependencies(cur_package_dict:dict, parent_packages=[]):
    global package_name_to_req_by
    global deep_req_by
    global package_name_to_reqs
    global deep_reqs

    cur_package_name = cur_package_dict['package_name']
    # managing logic of packages required by cur package
    if cur_package_name not in package_name_to_req_by.keys():
        package_name_to_req_by[cur_package_name] = []
    if len(parent_packages) > 0:
        if deep_req_by:
            package_name_to_req_by[cur_package_name].extend(parent_packages)
        else:
            package_name_to_req_by[cur_package_name].append(parent_packages)

    # managing logic of packages that cur package requires (i.e., dependencies of current package)
    if cur_package_name in package_name_to_reqs.keys():
        return package_name_to_reqs[cur_package_name]
    if len(cur_package_dict['dependencies']) == 0:
        package_name_to_reqs[cur_package_name] = []
        return []
    
    # setting deps. of cur package to top level packages
    top_level_deps = [dep_package_dict['package_name'] for dep_package_dict in cur_package_dict['dependencies']]
    package_name_to_reqs[cur_package_name] = top_level_deps

    if deep_req_by:
        child_will_be_req_by = [cur_package_name] + parent_packages
    else:
        child_will_be_req_by = cur_package_name

    for dependency in cur_package_dict['dependencies']:
        # recursively adding top level deps' deps if deep_reqs is True, otherwise, just execute req_by and reqs logic on next child (i.e., dependency)
        if deep_reqs:
            package_name_to_reqs[cur_package_name].extend(add_dependencies(dependency, child_will_be_req_by))
        else:
            add_dependencies(dependency, child_will_be_req_by)

    # all return statements of this function are useful only if deep_reqs is set to true        
    return top_level_deps

# JSON data
json_data = [
    {
        "key": "aiohttp",
        "package_name": "aiohttp",
        "installed_version": "3.8.3",
        "required_version": "3.8.3",
        "dependencies": [
            {
                "key": "aiosignal",
                "package_name": "aiosignal",
                "installed_version": "1.2.0",
                "required_version": ">=1.1.2",
                "dependencies": [
                    {
                        "key": "frozenlist",
                        "package_name": "frozenlist",
                        "installed_version": "1.3.3",
                        "required_version": ">=1.1.0",
                        "dependencies": []
                    }
                ]
            }
        ]
    },
    {
        "key": "hadooken",
        "package_name": "hadooken",
        "installed_version": "3.8.3",
        "required_version": "3.8.3",
        "dependencies": [
            {
                "key": "aiosignal",
                "package_name": "aiosignal",
                "installed_version": "1.2.0",
                "required_version": ">=1.1.2",
                "dependencies": [
                    {
                        "key": "frozenlist",
                        "package_name": "frozenlist",
                        "installed_version": "1.3.3",
                        "required_version": ">=1.1.0",
                        "dependencies": []
                    }
                ]
            }
        ]
    }
]


{'Bottleneck': ['numpy'],
 'Flask': ['click',
           'importlib-metadata',
           'itsdangerous',
           'Jinja2',
           'Werkzeug',
           'colorama',
           'zipp',
           'MarkupSafe',
           'MarkupSafe'],
 'Flask-Compress': ['brotli',
                    'flask',
                    'click',
                    'importlib-metadata',
                    'itsdangerous',
                    'Jinja2',
                    'Werkzeug',
                    'colorama',
                    'zipp',
                    'MarkupSafe',
                    'MarkupSafe'],
 'Jinja2': ['MarkupSafe'],
 'MarkupSafe': [],
 'Pillow': [],
 'PyQt5': ['PyQt5-sip'],
 'PyQt5-sip': [],
 'PySocks': [],
 'PyYAML': [],
 'Send2Trash': [],
 'Werkzeug': ['MarkupSafe'],
 'absl-py': [],
 'aiohttp': ['aiosignal',
             'async-timeout',
             'attrs',
             'charset-normalizer',
             'frozenlist',
             'multidict',
             'yarl',
             '

In [34]:
import subprocess

def get_pip_package_list():
    command = ['mamba', 'list']
    result = subprocess.run(command, capture_output=True, text=True)
    output_lines = result.stdout.strip().split('\n')[2:]  # Remove header rows

    package_list = []
    for line in output_lines:
        if line.count('pypi') >= 2:
            package_name = line.split()[0]
            package_list.append(package_name)

    return package_list

['fastdup', 'image-to-numpy', 'pipdeptree', 'simple-hierarchy', 'torchaudio', 'torchvision']


In [35]:
import argparse

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('-p', '--packages', nargs='+', required=True, help='passing x package(s) will output x requires/required-by descriptions')
    parser.add_argument('-dd', '--deep-dependencies', help='recursively output all packages which package(s) requires')
    parser.add_argument('-drb', '--deep-required-by', help='recursively output all packages which depend on package(s)')
    parser.add_argument('-spi', '--show-package-installer', help='in the output, displays the packages installed with pip vs mamba (conda)')
    args = parser.parse_args()

    result = subprocess.run(['pipdeptree', '--json-tree'], capture_output=True, text=True)
    result_json = json.loads(result)

    # Set to store package dicts
    package_name_to_reqs = dict()
    package_name_to_req_by = dict()
    deep_reqs = False if args.deep_dependencies is None else True
    deep_req_by = False if args.deep_required_by is None else True

    # Traverse the JSON data recursively to fill in package_name_to_reqs and package_name_to_req_by
    for cur_package_dict in result_json:
        add_dependencies(cur_package_dict)

    if args.show_package_installer is not None:
        pip_packages_installed = get_pip_package_list()
    else:
        print('since -spi is not passed, will assume all packages are mamba/conda installed:')
        pip_packages_installed = []

    for package_name in args.packages:
        print('package name: ', package_name)
        print()
        print('dependencies:')
        print('pip installed:')
        pprint([pkg for pkg in package_name_to_reqs[package_name] if pkg in pip_packages_installed])
        print('mamba/conda installed:')
        pprint([pkg for pkg in package_name_to_reqs[package_name] if pkg not in pip_packages_installed])
        print()
        print('dependencies:')
        print('pip installed:')
        pprint([pkg for pkg in package_name_to_reqs[package_name] if pkg in pip_packages_installed])
        print('mamba/conda installed:')
        pprint([pkg for pkg in package_name_to_reqs[package_name] if pkg not in pip_packages_installed])
        print()
        print('required by:')
        print('pip installed:')
        pprint([pkg for pkg in package_name_to_req_by[package_name] if pkg in pip_packages_installed])
        print('mamba/conda installed:')
        pprint([pkg for pkg in package_name_to_req_by[package_name] if pkg not in pip_packages_installed])
        print()
        print()
    

    

usage: ipykernel_launcher.py [-h] [-p PACKAGES [PACKAGES ...]]
                             [-dd DEEP_DEPENDENCIES] [-drb DEEP_REQUIRED_BY]
                             [-spi SHOW_PACKAGE_INSTALLER]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9011 --control=9009 --hb=9008 --Session.signature_scheme="hmac-sha256" --Session.key=b"7297d98e-5fad-49fb-9738-ef4ba0f48409" --shell=9010 --transport="tcp" --iopub=9012 --f=c:\Users\ashra\AppData\Roaming\jupyter\runtime\kernel-v2-40080ctk089z515Zq.json


SystemExit: 2

d:\CS\projects\graduation_project\.mamba\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
